# Demo (in gradio)

In [10]:
# imports
import os
import json
from dotenv import load_dotenv
from typing import Optional

from summarize_media.get_media.youtube import get_youtube
from summarize_media.pre_processing.convert_audio_format import convert_to_wav
from summarize_media.transcribe.transcribe_cloud import get_transcribe_cloud
from summarize_media.host_files.host_files import upload_file_to_0x0
from summarize_media.post_processing.reformat_output import reformat
from summarize_media.summarize_transcription.summarize import get_summarization

load_dotenv()

True

In [11]:
# Hard coded for demo purposes
media_output_path = "data/media"
transcript_output_path = "data/transcript"
local_size_limit = 40 # in MB

In [12]:
def get_name(file_path: str):
    _, name = os.path.split(file_path)
    name, _ = os.path.splitext(name)
    return name

def pipeline(url: str, 
             sampling_rate: int = 16000, 
             save_results = False, 
             output_path: Optional[str] = None,
             cloud_transcribe_model_name: str = "large-v3",
             cloud_transcribe_kwargs: Optional[dict] = None):
    # Fetching Audio from youtube and converting to wav
    audio = get_youtube(url, media_output_path)
    name = get_name(audio)
    
    audio = convert_to_wav(audio, sample_rate=sampling_rate)

    # Directly supply the file for inference or upload the file depending on size
    size = os.path.getsize(audio)
    print(f"Converted audio size: {(size / 1024) /1024:.1f} MB")
    if size < local_size_limit * 1024 * 1024:
        print("Upload file directly to inference endpoint")
        audio = open(audio, "rb")
    else:
        print("Exceeded size limit, uploading to file host before performing inference")
        audio = upload_file_to_0x0(audio, 3600)
        print("Uploaded File")
        

    # Transcribing and reformating the audio
    print("Starting transcription process")
    cloud_transcribe_kwargs = cloud_transcribe_kwargs if cloud_transcribe_kwargs else {}
    transcript = get_transcribe_cloud(audio, 
                                      model_name=cloud_transcribe_model_name, 
                                      **cloud_transcribe_kwargs)
    transcript = transcript["segments"]
    transcript_txt = reformat(transcript)
    summary_txt =  get_summarization(transcript_txt)
    
    output_path = output_path if output_path else transcript_output_path
    transcript_json_path = os.path.join(output_path, name + ".json")
    transcript_txt_path = os.path.join(output_path, name + ".txt")
    
    if save_results and output_path:
        with open(transcript_json_path, 'w') as file:
            json.dump(transcript, file)
            
        with open(transcript_txt_path, "w") as file:
            file.write(transcript_txt) 
        
    return summary_txt, transcript_txt


In [ ]:
def pipeline_local_file(file_path: str, 
                        sampling_rate: int = 16000, 
                        save_results = False, 
                        output_path: Optional[str] = None,
                        cloud_transcribe_model_name: str = "large-v2",
                        cloud_transcribe_kwargs: Optional[dict] = None):
    
    name = get_name(file_path)
    audio = convert_to_wav(file_path, sample_rate=sampling_rate)
    cloud_transcribe_kwargs = cloud_transcribe_kwargs if cloud_transcribe_kwargs else {}

    # Directly supply the file for inference or upload the file depending on size
    size = os.path.getsize(audio)
    print(f"Converted audio size: {(size / 1024) /1024:.1f} MB")
    if size < local_size_limit * 1024 * 1024:
        print("Upload file directly to inference endpoint")
        audio = open(audio, "rb")
    else:
        print("Exceeded size limit, uploading to file host before performing inference")
        audio = upload_file_to_0x0(audio, 3600)
        print("Uploaded File")
        

    # Transcribing and reformating the audio
    
    print("Start Transcription")
    transcript = get_transcribe_cloud(audio, 
                                      model_name=cloud_transcribe_model_name, 
                                      **cloud_transcribe_kwargs)
    transcript_txt = reformat(transcript)
    summary_txt =  get_summarization(transcript_txt)
    
    transcript_json_path = os.path.join(output_path, name + ".json")
    transcript_txt_path = os.path.join(output_path, name + ".txt")
    
    if save_results and output_path:
        with open(transcript_json_path, 'w') as file:
            json.dump(transcript, file)
            
        with open(transcript_txt_path, "w") as file:
            file.write(transcript_txt) 
        
    return summary_txt, transcript_txt

In [5]:
from IPython.display import display, Markdown

url = "https://www.youtube.com/watch?v=-MFJCmHyFvM"

summary, transcript_txt = pipeline(url, save_results = True)

display(Markdown(summary))

Title: New company filters embryos for IQ and Height  Genetic determinism and Eugenics
Upload date: 14/11/24 12:45:00
Time length: 0:23:33


INFO:summarize_media.host_files.host_files:Uploading file: New company filters embryos for IQ and Height  Genetic determinism and Eugenics.wav


Converted audio size: 86.2 MB
Exceeded size limit, uploading to file host before performing inference


INFO:summarize_media.host_files.host_files:Successfully uploaded to https://0x0.st/XdpW.wav
INFO:httpx:HTTP Request: POST https://api.replicate.com/v1/predictions "HTTP/1.1 201 Created"
INFO:httpx:HTTP Request: GET https://api.replicate.com/v1/models/victor-upmeet/whisperx/versions/84d2ad2d6194fe98a17d2b60bef1c7f910c46b2f6fd38996ca457afd9c8abfcb "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


## Transcript Summary
A detailed discussion about IVF (In Vitro Fertilization), embryonic screening, and future implications of genetic selection technology, focusing on a new startup called Heliospec Genomics and their controversial services.

## Key Talking Points
* **Basic IVF Process** (00:00:31)
  - Extraction of eggs
  - Fertilization in laboratory conditions
  - Creation of embryos
  - Implantation

* **Embryonic Screening Technology** (00:01:57)
  - DNA assessment capabilities
  - Use of genomic databases
  - Screening for traits like IQ, height, and health conditions

* **Heliospec Genomics Case Study** (00:03:20)
  - Charging up to $50,000 for 100 embryos
  - Claims of 6+ IQ point improvements
  - Using UK Biobank data
  - Future plans for screening personality traits

## Main Themes
1. **Technological Advancement vs. Ethics**
   - Questions about genetic enhancement
   - Accessibility concerns
   - Social implications

2. **Future Societal Impact**
   - Potential class divide
   - "Bifurcation of human race"
   - Concerns about artificial conception

3. **Personal Skepticism**
   - Speaker's reservations about lab-grown eggs
   - Philosophical concerns about "natural" vs. artificial conception

## Context Notes
- References to Nietzsche's concepts of "last man" and "Ubermensch"
- Discussion assumes knowledge of basic genetics and IVF procedures
- Contemporary context of wealth inequality and technological access

## Source Information
- Duration: Approximately 23:32 minutes
- Format: Audio/Video transcript
- Speaker: Single primary speaker (informal, commentary style)
- Context: Educational/Commentary content

Note: The speaker uses informal language and some crude terminology throughout the discussion. Some technical claims about genetic prediction and outcomes are presented as speculative rather than scientifically verified.

Test Summarization prompts

In [1]:
text = """
Start: 00:00:00 - End: 00:00:27
 video for a minute apparently it's about hagmaxing i know nothing about it um again i'm old so i've learned new terms almost every single fucking day every single fucking day so this should be a good one social and economic factors and the rich evil let me know if you can hear this i think it should be good government the american dream of owning a house and having a wife and two kids is dying and the zoomer coomer fantasy of a rich older girlfriend taking charge and making you chicken nuggies while you play video games in your pajamas all day is
Start: 00:00:28 - End: 00:00:58
 I just want to just comment on that really fast. I probably brought this up a couple of times because I'm a broken record. But as you guys know, I used to be married before and, you know, kids and everything. So it's funny because now that I'm out of it, I think to myself, like, do I still want stuff like that? And hearing that alone, I'm just like, hmm, it seems like the whole culture nowadays is kind of like pushing away from that. I don't know how I feel about that. You know, I have like mixed feelings. I have mixed feelings.
Start: 00:00:58 - End: 00:01:28
 But the whole thing about older people. Okay. So at least from my understanding, hag maxing is young folk wanting to date older people. That's my understanding. I'm sure she'll tell me. But chat, you're going to have to explain some of this for me because you know me. I don't. I have no idea. I have no idea. Thriving. Back in my day, we called this cougar hunting. Same. Same. Mommy issues. Also same. Still have those.
Start: 00:01:28 - End: 00:01:57
 The kids today are calling it something different, hag maxing. I have been told, also I think this might be just a tad bit too loud. I have been told that there is a lot of things now where it's maxing. Everything's like maxing, hag maxing, game maxing, or not game maxing, game, game. Apparently there's a few things. Apparently there's a fucking few things. I'm not exactly sure if that's like actually a thing or not, but chat, I'm sure you guys can explain all that to me. Actually, you know, before I continue,
Start: 00:01:57 - End: 00:02:24
 Is that just like a thing now? It's just like everything is maxing. Who made that? How long has this been a thing? I gotta know. Cause apparently I'm missing out on a few things. Apparently I'm missing out on like a few things. I'm assuming this is semi-recent cause I'm just barely hearing all this. It's heavy maxing. Thank you. Thank you. Um, I think the very first time that I heard any of this shit was probably in February when it was my birthday. Um, we just hit 31.
Start: 00:02:25 - End: 00:02:53
 And a couple of people came in. I think someone rated into me. They're like, oh my God, you're 31. You're finally a hag. You're finally like hag maxing. I never even asked what the fuck that meant. I had no idea what the fuck that meant. So I just rolled with it. I just rolled with it. Because when I think of hag, I think of a witch, which by the way, it is October. So I was like, oh, perfect timing. Perfect timing. But apparently it is not that. Then again, I could go as a witch this year for Halloween, but it doesn't matter. It doesn't matter.
Start: 00:02:54 - End: 00:03:23
 In my last video, I jokingly referenced the term hag-maxing. You've been on a date with a 40-year-old? Okay, that's different. How is that different? He's right. Remember how I said that I will only watch two videos with her? That was the first one. That was the first one. I think I might have watched that with Asmin when he was reacting. I think that's the one. Really good video. I think that was about dating, especially nowadays. So, yeah. Things have been changing a lot in the dating sphere.
Start: 00:03:23 - End: 00:03:52
 Now, I haven't been on a date or done all that shit in the years, which really I don't want to. But every time I see more and more of this shit, I'm just like, oh, maybe I just stay away. Maybe I just stay away. It's different. You're robbing the cradle. He's hag maxing. Educate yourself. Which greatly intrigued 37. Wait, chat, you know, I gotta ask you, what is the age? What is the age that everyone's like, oh, this is when it actually starts. I was told that it's after 30. That's what I was told.
Start: 00:03:54 - End: 00:04:23
 Oh wait, so it's 30, 18. Shut up. Shut the fuck up. You're just barely getting out of the cradle. 30, 30, 30. Okay. I thought it was after 30. Well, must've just already been like that. You know, actually another thing is, and I'm sure she'll say it, but I have noticed an extreme want for older women as of late. You know, last I checked, last I checked in like the real world, um, that wasn't really a thing. Must just be like an internet thing. Must be, must be.
Start: 00:04:24 - End: 00:04:54
 A lot of you. So I decided, you know what? I am going to make an entire video dedicated to covering this phenomenon. Gen Z men going out of their way to date older women. Chat, is that any of you? Actually, you know what? Actually, I don't want to know. I don't want to know. I don't want to know. What is this? Nevermind. Nevermind. Nevermind. Nevermind. Is this happening? And if it's happening, is it a good thing or is it a symptom of broader societal issues? Why are all these young whippersnappers scooping up these older time pieces?
Start: 00:04:55 - End: 00:05:21
 Oh, speaking of time pieces. Before she continues with that, I can tell you why the opposite. I can tell you why the opposite. With my ex-wife, she was older than me by six years. It is a couple of things. One, it's definitely a sex thing. I will say that. I will say that. Another thing is more just being with someone with more energy.
Start: 00:05:21 - End: 00:05:45
 Um, sometimes when someone's older, like guys, girls, doesn't really matter. Um, a lot of the times, a lot of the times, um, you'll start drifting. Someone may have energy. Someone may not, um, in a normal guy, girl relationship. I was with girl, girl, but of course, but like for me, like that was a little different, but from what I've heard from normal relationships, um, I think, um,
Start: 00:05:45 - End: 00:06:13
 So women tend to get in their prime more in their 30s. You know, that's when you really get inside their prime. That's when you get a little bit, that's when you get a little rugged. Now guys start to slowly slim away from it all. Like kind of, what is it? Your libido or something. I forget. I forget. Don't mind me. I just barely woke up. So bear with me. But point I'm trying to make is that the sex drive and everything starts to slowly slim while women's actually increase. It's about in their 30s. So sometimes you won't see eye to eye with things.
Start: 00:06:14 - End: 00:06:39
 That's why. And again, I don't want to, I don't want to scare nobody, but that's why you'll see a lot of people either like they'll have affairs or anything else, usually in their thirties. Not sure if you ever noticed that, but tends to happen. Not everyone clearly, clearly, but there's a lot of issues that tend to happen during that time. Plus like a lot of times marriages have been there for a few years already. So you'll have a lot of differences. Then eventually just
Start: 00:06:39 - End: 00:07:08
 That's what I've seen. That's usually what I've seen. Now, hopefully it doesn't happen to anyone here that's actually married. But that's what I've noticed from my scientific calculations. Hopefully I'm wrong with that, but that's what I've seen. Oh, it's right inside of an ad. Is anyone actually... Who actually still has watches? I used to. I used to have... What is it? G-Shocks.
Start: 00:07:09 - End: 00:07:38
 I used to have G-Shocks back when I was active duty because here's the thing. They never broke. You can just jump in the water. You can do all that. You can smash it. It didn't really matter. It was just good. Yeah. But who still has watches? Anyone actually use them still? I just feel like watches are like something from the past. I had to take mine off for work. I know G-Shocks were great and they really weren't all that much. They really weren't all that much. I just would have it because like, listen, being an engineer, you tend to get your, um,
Start: 00:07:39 - End: 00:08:07
 you will definitely get your hands dirty. And also you just smash things, smash things like with your hands on accident. So it's nice to have something where it's not going to break. So that was always nice for me. Yeah. And here's the thing. I liked watches because I didn't have to pull out my phone. Like it was just that simple, especially if you're in like a crawl space or whatever, whatever. So really depends on the situation. I prefer to use my phone, but usually I pull out my phone when I don't need it. And it just, it would just break. I can't tell you how many times I broke a phone.
Start: 00:08:08 - End: 00:08:28
 I cannot tell you how many times I broke a fucking phone. And women. Personally, I'm a huge fan of their men's watch collection. Easy. I completely agree with you. I like to have that. But so far, we're good. Fine grained walnut wood. So far, we are good. Stainless steel. The walnut wood comes from here in North America. And all the wood Holzkern uses are FSC.
Start: 00:08:28 - End: 00:08:57
 Does anyone have these watches? They look nice. They look nice. This is what I would get for like a boyfriend or something. Maybe, maybe. Then again, if I got one for a boyfriend, which again, that's not going to happen for a minute, but hypothetically, yeah, I don't know. I don't know. I would have to like, if you all fancy and proper, maybe if you worked in the business world, like how I do, maybe, but really I would try to get them something more practical, something that we both like.
Start: 00:08:57 - End: 00:09:27
 Maybe. Maybe, maybe. Feels a bit annoying to you? Yeah, it just, I don't really like things all over me. Even though I have a few. Even though I have like a few. I have a few. Just saying. The watch I picked out for myself is the Colorburst Automatic Watch. It has a unique green mosaic embedded in a stainless steel casing. And its wooden strap is easily adjustable to fit every wrist. Holtzgren's products are made in small quantities to ensure that each item
Start: 00:09:29 - End: 00:09:58
 They do a great job combining functionality and fashion. And I just think their watches look... We could skip ads and stuff, but here's the thing. I like supporting other fellow content creators, so I could care less. But they're really neat. And besides, I'm learning about watches, so shut up. Earrings, rings, bracelets, and necklaces. Like this platinum-plated shade necklace. The store has accessories for both men and women. Like the durable Forte bandlet for men. I haven't worn jewelry in forever. Or the duet bandlet for women. Even though I have this right here.
Start: 00:09:58 - End: 00:10:27
 Like in real life. Like in real life. My IRL life. Oh, they're kind of expensive though. Hmm. If I remember correctly, when I was getting G-Shocks and this was like, I don't even know, like 10 years ago or something. $69 or something. I think. I think it's like $69. I could be wrong. They do look nice. That is true. That is true. That is true.
Start: 00:10:27 - End: 00:10:54
 Because I was about to say, some of that stuff can be a little, you know, kind of cheap. It can be kind of cheap. Also, damn, I love that emote. So cute. So cute. Is that a cat? Adorable. Adorable. Sorry, I like cats. More of a doll person, less like cats. I do like them, though. I just like the wooden box that they're in.
Start: 00:10:55 - End: 00:11:23
 Sorry, I got overly excited for freaking watches. I will say one thing. I find some of these freaking, these new words so stupid. But then again, I'm at that age now. I'm at that age now where I'm like, back in my day, our words were so much better, even though ours were like swag and... God, wait a minute, wait a minute. It was swag, it was...
Start: 00:11:23 - End: 00:11:51
 Well, she said a few of them. I'm just thinking like, what else? What else? I'm sure there's more, but listen, I don't want to go down memory lane. I do remember swag. I don't know why that one was so big for a while. It was, it was, it was huge. YOLO. Oh my God. No. Okay. You know what? We're not going down this. We're not going down this. We're not going on this. You're going to, oh God, I'm 31 already. You're going to make me feel like I'm 51. I already kind of fucking feel it. Okay. I'm not looking at chat. I'm not looking at chat.
Start: 00:11:52 - End: 00:12:19
 you guys i'm so tired of comments that are like i used to watch you when i was in third grade and now i'm graduating college how old are you wait hold on you look like in your like maybe your mid-20s is she my age no way i'm not saying i look old i'm not saying i look old but she looks like she is like in her mid-20s crazy okay interesting i like you already this is like my second video watching of her so nice
Start: 00:12:20 - End: 00:12:50
 Sorry, I don't mean to pause, but that's awesome. That's awesome. My knees hurt when I get up off the floor. That's how I feel. I blame that on the fucking military. I blame that on the fucking military. Gen Z's new social trend of dating older women. I thought this is the age. A very strange phenomenon is making its rounds on social media. Gen Z men are ditching casual sex with their women to pursue long-term relationships with financially stable and experienced millennial ladies. Now, even though this article doesn't actually exist, it was a meme created on 4chan because of course it was.
Start: 00:12:50 - End: 00:13:19
 Never been on a 4chan, but looking at that, looking at that, one thing I do want to say is this. In my current situation, now clearly I'm not dating, but one thing I will say is that in my life right now, if there's somebody that I like, if there's someone that I can really vibe with, right? I'd like to have someone that is on the right path. But at the same time, let's just say they're going through school or college. Do they have money right now? Absolutely fucking not. But I'm at that point where I don't care.
Start: 00:13:20 - End: 00:13:47
 I don't care as long as they're kind of on the right track. If that makes sense. Now, if I was younger, I'm like, whoa, listen, we're both broke. So we're gonna have to do something. But I think I'm in a better spot now. Like, again, I work two jobs. I do streaming. I do it every like, you know, I do a lot of fucking shit. I do a lot of shit. So really money is not as bad as it once was as it once was. So I can understand where someone like having to deal with somebody that is more exciting and
Start: 00:13:47 - End: 00:14:17
 exciting somebody that wants to do stuff feels alive feels like they're not like slowly dying because i know i am sometimes it's nice to have it's nice to have all right but yeah so that's i can i can kind of see it for that but i'm sure people do it for other reasons but of course oh and i just went backwards massive conversation and debate about hag maxing and people had a lot of strong opinions about it motherless behavior
Start: 00:14:20 - End: 00:14:50
 That is a great first message. I mean, I bet you some of the people it's true. Sorry, sorry, sorry. This is just mommy issues rebrand. Another day, another psyop. It's not ha- Chat, what does psyop mean? I've heard that for military. Doesn't it just mean crazy? Chat, you know I have no idea what some of these fucking terms mean. That just means crazy, right? You would think I know because that is a, that is definitely a military term. I'm going to pretend like I haven't been in the military for fucking 12 years.
Start: 00:14:52 - End: 00:15:19
 Okay. See, I was about to say, but what does it mean for just regular shit? Just regular shit. Mind games. Mind control. Oh, when someone's trying to like fuck with you. Oh, okay. That makes a little bit more sense. That makes a ton more sense. Okay. Okay. I was always wondering what that meant. Cause I seemed like someone's like, man, they're saying like this way. Actually, I can't give you like an example, but I've heard it around like before.
Start: 00:15:20 - End: 00:15:46
 well yeah i know what mk ultra is i just didn't know what it meant in just like the normal world the normal world whenever people are trying to use it here i just thought that meant that someone was crazy so tag maxing it's milf hunting and it's an art please don't misappropriate my fetishes giving the dumbest name known to humankind to having a mommy kink i mean to be honest i'm 16 waiting for a 20 to 25 year old to pick me up you need to live life first
Start: 00:15:46 - End: 00:16:14
 You need to live life just a little bit, just a fucking little bit. Please and thank you. Just a little bit. I don't know. Go to your, get to 20 first. Sure. I'm telling you, most of the times, most of the times when I was dating guys, I would say that they kind of mature a lot more in 25, 24, 25, somewhere around there. Now everyone's different clearly, but that's usually the range I start seeing a little bit more.
Start: 00:16:15 - End: 00:16:43
 kind of kind of really it's more in their 30s but well you know i'll take what i can get usually with that that that's like the general that's a general basis at least for me i'll sign crim thank you for the raid i really appreciate that thank you lol okay officer i'm gonna throw up i would love to have a 40 year old i've got a great stream thank you so much for stopping by bills drizzle drizzle young hard dicks and the older women who enjoy them this is this is what i mean this is what i mean
Start: 00:16:45 - End: 00:17:13
 It's fresh. It's fresh. You have somebody that knows what they're doing, usually. Mid-25, you usually feel like, you know, they're a pretty strong buck. Let's say that. That's why a lot of the times people will tend to go for them. 35, 36, going in with a 25-year-old. Yeah, it sounds about right. Sometimes it's just a one-night stand. Sometimes they keep them around. Sometimes they keep around a little bit. Sometimes it kind of keeps them on a side piece. Sometimes it's like the whole shebang. It really just depends. Everyone's different. This isn't new.
Start: 00:17:13 - End: 00:17:32
 Some people are married with that, so. But listen, we'll get to that another time. Bandwagoners are so cringe. We'll get to that another time. I've been hagmaxing since second grade in Miss Johnson's class. I am going to kill every last Gen Z and Gen Alpha kid on this planet, including myself. Fair enough.
Start: 00:17:33 - End: 00:18:02
 My thought process on that is this. Do I think it's strange if someone's like in the 40s trying to date like an 18 year old? Dr. Disrespect, I'm talking about you. But at the same time, if there's certain spots where it's legal, sure, whatever.
Start: 00:18:02 - End: 00:18:31
 Now I do think to myself, again, being a mom, I'm just thinking to myself, like, okay, if my young daughter is talking to a guy that's in his forties, maybe like older than me, we're going to have some problems. I'm just saying, I rather not have that double standards. Okay. Listen, listen, I don't care either way. You know what? Do you, if you're an adult, you're in a fucking adult. Um, at the same time, do I feel like you should like maybe kind of talk with someone near your age, at least a few years between that? Yes.
Start: 00:18:32 - End: 00:18:59
 10, 15 years. Maybe not so much. Hey, trust me, chat. I'm with you guys with the whole like double standard thing. Stupid, stupid. Listen, I like to keep it fair. I like to keep it fair. Makes sense to me. I'm not one of those people where girls can do this, but not guys. I find that so fucking stupid. If you're an adult, you're an adult. You want to hang out with other adults, go for it. Even though I feel like it's a little strange if you're like 15 years apart or something, but it is what it is. It is what it is.
Start: 00:19:00 - End: 00:19:29
 Ah, I f***ing love adult women. Women over 32 should be illegal. I still got one year left, baby. I still got one year left. High five, Chad. Awesome. Now, according to the very astute. I still got one year left, bitch. Knowyourmeme.com. The term hag maxing is very popular in the VTuber community. Now you chat. Ain't no f***ing way. Ain't no f***ing way.
Start: 00:19:29 - End: 00:19:57
 You know what? Actually, that checks out. That checks out. That checks out a lot, actually. Hmm. Y'all are stupid. Y'all are stupid. Where do they even get that info from? Well, to be honest with you, just sit in a couple of streams. Just a couple of streams a couple times. You'll see it. You'll see it. I don't have much interest in the VTuber community. Don't worry, me too. But they seem to have a lot of interest in me. A lot.
Start: 00:19:58 - End: 00:20:25
 of interest should i do a video reacting to vtubers reacting to me one day is this just like a person sorry i'm gonna pause a lot is this just a person that like vtubers like to do videos on like i said again this is my if i want to say like our reactions to our very first one with her i like her i like her energy i like her energy i see i see hey listen this is my first one okay listen go to the youtube channel you'll see you'll see
Start: 00:20:25 - End: 00:20:51
 Leave me alone, Chad. That might be fun. Hacks stick together, true and real. Although it seems to be a meme amongst the VTubers and the weebs, it is apparently a thing happening in real life. Rise of the boy toy. I can't believe she said VTubers. Yeah, that's you, Chad. That's you. Y'all need to fix your shit. Younger men seem more open and less burdened by insecurities.
Start: 00:20:52 - End: 00:21:18
 Okay, I have a feeling I'm going to be saying this like a hundred times in this video, but could you imagine an article like this with the genders reversed? It's always like that too. It's always like that. And again, that's what I was saying with the whole like double standard thing. I don't like it. Trusting chat. I'm right there with you. I'm on your side. I'm on your side. I fucking hate that shit too. When someone's like, oh, well I can do this, but not you. It's like, why? It's like, oh, I didn't know that the law changed for us.
Start: 00:21:18 - End: 00:21:42
 just because i have a fucking pussy and not a fucking dick i don't know listen i didn't read that in the laws no no okay strange i love dating strange boys they're less bitter they have less baggage they're not ran through people would just be calling it red pill they're not ran through age gap relationships between millennial women and when i think of that i think of that damn meme which i try to bring it up like um i think it was like a couple streams ago
Start: 00:21:43 - End: 00:21:58
 Where it's that one meme where the girl's like, all right, I'm ready to settle down. And her pussy looks like fucking roast beef. Looks like an RV sandwich. I want to say that's what she's referring to. I pray that's what she's referring to. And if she is, she's my new best friend.
Start: 00:21:59 - End: 00:22:29
 the rise our analysis of age hypogamy reveals the pragmatic underpinning of these relationships it sometimes comes down to who's available many older men prefer to date younger women the same is true for a lot of older that is something i've been noticing a lot gen z men emerging as appealing dating options for their part many gen z men might have few opportunities to date women their own age i want to comment on that so something i noticed because we've been doing a lot of videos on what do we do we've done like only fan stuff we have done
Start: 00:22:31 - End: 00:22:59
 We've done, okay, let's just say this. We've done a few videos on like more or less people spawn camping girls that just barely hit 18. Then they go off to their OnlyFans. And then like, I'm just like thinking to myself, like, aren't you like 40, 45? You're just waiting for these young girls hit 18. And I'm just like, why y'all do that? I mean, don't get me wrong. Don't get me wrong. Younger, I understand, blah, blah, blah. But like that young, that young. Chat, listen, someone explain that to me. Someone explain that to me.
Start: 00:23:00 - End: 00:23:29
 Because I'm just like, don't you kind of like it? Like a little bit, a little bit older. Because listen, from my experience. Okay, so I have told this story before. My quote unquote COVID mistake. She was a lot younger than me. So I think I was like 27. And she was like 21. So it was about six years apart. That was actually the same age that me and my ex-wife met originally. And I was like, all right, I'll give it a shot.
Start: 00:23:30 - End: 00:23:59
 Um, it was different. And the reason why I say that is because we were in two different parts of our life. She was just barely even able to like get alcohol for the first time by herself, which is strange, but like she slowly became an alcoholic. And for me, I wasn't trying to do none of that shit. I wasn't trying to do none of that shit. That was just not for me. That was not for me. I was like, I've already been that done that taking fucking pictures. Then again, after duty, you have this tendency to drink a lot. So it was.
Start: 00:24:00 - End: 00:24:24
 was a bit of a mess it was a bit of a mess but like that's why i tell people like it's all about the vibe it's all about what you guys have been a part of but we were in two different parts of our life so we fought a lot so that's why when i think of someone that's really old and super super young i'm just like there's no way you all are gonna match there's no way like it has to be just for a quick it has to be and listen if that's what you want to do fine
Start: 00:24:25 - End: 00:24:55
 But like, if you're trying to like, you know, there are people that are like, oh, we can do this together as like a relationship. I'm like, can you though? I would be surprised. I would be fucking surprised. I don't know. I don't get it. Unfortunately, that's been a thing forever. Yeah. I can only imagine. I can only imagine. Like people are interesting. People are interesting. Also what I've never seen a website like that. Listen, I've been off the internet for like forever. Listen, the last three years I've been learning all about stuff.
Start: 00:24:55 - End: 00:25:17
 Let's just say that. Streaming, how people do things on the internet. Sometimes I wonder, was this a good idea? I have no idea. Anyway, I have fun with streaming. So let's just say that. So all is good here. Hi, Whiskey. How you doing? Am I winning? Kind of. I'll be having an awesome day.
Start: 00:25:18 - End: 00:25:48
 If you're an older millennial woman, you might be attracted to younger men for their energy, inclusive views about sex and gender, and superior care and literacy about their partner's emotional well-being. So is that why you guys like hanging out with me? Because I'm starting to see something. Energy. I care about you guys. I'm just like, oh, okay, I can kind of see that a little bit. I can kind of see that just a little bit. A little bit. I always wonder why some people watch my old ass and I'm like, hmm, it
Start: 00:25:48 - End: 00:26:16
 It's a possibility. It's a possibility. There are also older VTubers that are out there too. So I can, maybe that's what it is. If we're all still kind of giving out both the same vibes, then I can understand that. I get it. I get it. Now, my question is this, is it more like an actual relationship or does like, do the Zoomers want us to take care of them? Like, like, like little babies. It's almost as if like, you want me to treat you like a little baby, get you food, but also have sex afterwards.
Start: 00:26:16 - End: 00:26:41
 i want to say it's kind of like that i want to say it's kind of like that like i ever i always wonder if like maybe like they want to also be a part of the day shift i don't know kind of contribute maybe not she did say something about playing video games and eating chicken nuggets so maybe not maybe not it's hard to say it's hard to say either way it's uh it's interesting let's say that
Start: 00:26:42 - End: 00:26:48
 In many ways, Gen Z men see millennial women as an opportunity to level up emotionally and socially.
Start: 00:26:49 - End: 00:27:16
 With their evolved attitudes about everything from non-monogamy and gender fluidity to the anti-hustle soft life, Gen Z men may be the most likely generation to overturn powerfully entrenched stereotypes about sex and relationships. The kind featured by podcast bros. I love how millennial women have just psyoped themselves into believing a man who thinks f***ing a whole bunch of different women at the same time with no romance. Wait, hold on.
Start: 00:27:19 - End: 00:27:48
 i love how millennial women wait wait wait so what are they trying to say what are they trying to say i feel like i completely missed that sorry i think i just had a brain fart wait are they saying like okay wait somebody explain that to me somebody explain that to me what does that even mean see that's what i'm saying i was like wait a second wait a second i'm missing this it's like one of those big red pill talks oh okay okay so when they mean like when they mean like podcast bros they mean like
Start: 00:27:48 - End: 00:28:17
 andrew tate and shit i'm assuming so i'm assuming so i have no idea honestly i'm assuming so it's just another fucking psyop yeah you know what probably probably a man who thinks f***ing a whole bunch of different women at the same time with no romantic commitment is an evolved way of looking at relationships like yes queen that zoomer you're banging is so open-minded and progressive he's not like those podcast bros
Start: 00:28:18 - End: 00:28:44
 i swear modern i get it now they're talking about people that like praise us it's the whole slay queen model i fucking hate that shit i fucking hate that shit that whole culture to me personally is first off it's fucking toxic it's literally making it so it's like oh you can do whatever you want your body your fucking mind and don't get me wrong you can do whatever the fuck you want sure sure sure but it makes it so like everyone else's fucking shit
Start: 00:28:45 - End: 00:29:13
 They always say like, guys are fucking shit. And it's like, no, no, no. It's like empowering. But at the same time, really, I think it's just so that people can do whatever they want and without any consequences. And if people say like, otherwise, they're like, hey, like, hey, don't tell me what to do because blah, blah, blah, blah, blah. Like, you know, freedom and power and whatever, whatever. That's kind of how I see it. I think the whole culture is stupid, honestly. I think it's really stupid. I mean, granted, you could do whatever the fuck you want. But at the same time,
Start: 00:29:13 - End: 00:29:40
 Like, I think they use that excuse so they can do whatever. And usually a lot of times it's stuff where, like, you have to look at them, like, really? What is this? Also, hi, Private. Thank you so much for the rate. I really appreciate that. Also, hi, Mako. How are you doing? Good to have you guys. Exactly.
Start: 00:29:40 - End: 00:30:08
 exactly exactly it's like i love that took the words right out of my mouth genius even hollywood seems to be hopping on the hag maxing train why are so many older women dating younger men in recent rom-coms i'm old enough to be your mother size anne hathaway as she passionately entwines with amazon's new heartthrob nicholas galzin in the idea of you oh anne hathaway i'm sorry who wouldn't hag max with anne hathaway
Start: 00:30:08 - End: 00:30:35
 My papa loves her. My papa loves her so fucking much. So you listen, I see the appeal. I see the appeal. I like her energy. That's always my thing. She always has like a very like happy face. Happy face. Like, I don't know. I like energy. I like passion. I love energy. That's what gets me personally. That's what gets me. People that kind of feel like they kind of give a fuck. Kind of like they give a fuck. I don't know. I don't know.
Start: 00:30:38 - End: 00:31:02
 My goodness. Wait, wait, wait, wait, wait. Is that how? Okay, wait a second. Wait a second. Is that how you want me to talk to you guys? They're like, look how big you are. Look how big you are. It's like, wow. So look at your penis. It's huge. No one ever said that to you.
Start: 00:31:02 - End: 00:31:28
 I really don't care about this shit. Just kidding. Just kidding. It is so funny how much of this equality stuff is just, okay, now it's women's turn to do the thing we said was bad. All the new strong female leads in movies that are just kind of assholes celebrating of billionaire female CEOs. Like, did you actually want change or was it really all just penis envy? What it really is, is that power. It's power.
Start: 00:31:28 - End: 00:31:58
 People love power. People love to be able to be like, hey, I can do this and you can. I'm in a position that's better than you. Now, granted, guys, especially in the workforce, guys have always been above a lot of women. Now, we're starting to get to that point now where women are starting to do that, but it's not like equality. Start with equality to make it so that you can build yourself back up. Then it's a power thing. Literally start to become like the same thing. That's all it is. Swapping around, swapping roles, and then now they're inside power
Start: 00:31:59 - End: 00:32:26
 Again, when you abuse your power, you're just like everybody else, a piece of dog shit, piece of fucking dog shit. I don't care what, I don't care what freaking male woman, don't care, don't care. Like you're a piece of shit. You're a piece of shit. And you know, I think the equality thing, that seems to be where it starts to lead. I mean, again, am I wrong for that? I think so. That's what I've seen. And again, I worked in the business world for a very long time, still
Start: 00:32:27 - End: 00:32:51
 So that's what I see. That's what I see. Older women partnering with younger men. Why do some people have a problem with that? I think it's a power thing also. The French president's wife, Brigitte Macron, is almost 25 years older than her husband. See, that's kind of what I mean. I mean, it's not like they can't have things in common, but like.
Start: 00:32:55 - End: 00:33:22
 I mean, what specifically do you have in common when it's just got to be a sex thing? I mean, that's it. That's it. It's a young buck. That's it. It's a young fucking buck and an old bitch and an old hag. I mean, it just makes sense to me. It just makes sense to me. What else could it be? What do you want to talk to him and have sex with them and then make him play with his fucking cars? You want to play Legos afterwards? Like, I mean, I don't see it. I don't see it. I don't see it. I don't see it.
Start: 00:33:23 - End: 00:33:53
 They met when he was 15 and she was his high school teacher. Wait, that is not good. There should be a stigma against that. I agree with you. Maybe these high profile women will help shatter the stigma and normalize the benefits of widening our options when seeking romantic partnerships. After all, at every age, we deserve what our hearts desire. I'm sure there's a lot of people that said that and now they're in jail. I don't know. I feel like I've heard that before.
Start: 00:33:53 - End: 00:34:13
 hmm i don't see them anymore either it's strange it's strange it's almost as if it's it's almost as if like they went somewhere it's hard to say i don't know i don't know strange no no no when you were a teacher and your heart desires and don't even talk to me about like the ditty stuff oh god i've been researching all that
Start: 00:34:15 - End: 00:34:44
 yeah that's like a whole thing that's a whole video we'll talk about that one one of these days one of these days student you do not in fact deserve that what am i reading i'm 37 and my boyfriend is 23 i make upper six figures and own a condo in new york city at least three of my friends are also dating significantly younger men one is 32 and has been dating a 19 year old for three years wait hold on i don't rely on i'm sorry i'm sorry maybe i'm asking too much of your personal life chat but
Start: 00:34:45 - End: 00:35:14
 Anybody in a relationship like that. And if that's the case, you got to explain to me like how that started, how that started. Maybe that's not a video conversation, but at the same time, that is a lot. That is a lot. Yeah. That's like huge. That's huge. And I think about like me and my ex-wife with that. We were six years apart. We were six years apart. And I'm just thinking to myself like, all right, well, we had a lot of issues at first at the start.
Start: 00:35:14 - End: 00:35:43
 Because we were at two different parts in our life. I was a lot younger. She was more established. She had family. I did not. Then eventually I started to merge, but it took a long time. But that was still six years, not 19. To be financially stable. I don't need my man to have a good job. We both know what we want out of this relationship and we're getting it. You know, honestly, good for this. Wait, one is 32 dating a 19 year old for three. Oh, oh, she's 16.
Start: 00:35:48 - End: 00:36:13
 All right, so she met that guy when she was 16. You want to know what this reminds you of? Just probably worse. Just probably worse. Any military guys here, do you remember the creeps that used to like hang around the high schools and stuff? They're like, oh, I got this high school girlfriend. I'm like, yeah, but you're like 20 and you've already been in the military a couple of years. Can't you like, I don't know, go find somebody else?
Start: 00:36:14 - End: 00:36:42
 maybe go find somebody else i don't know i don't know i remember those people even when i was going off to high school it was uh just listen people are strange dating is strange how people think about like who they want to be with strange everything's strange everything's strange oh maybe because i became a mom and i had kids and i'm like no no we have a lot of love don't do that to my daughter like a man we are not drama queens we can't get pregnant we are very sexy
Start: 00:36:43 - End: 00:37:12
 I'm not like the other girls. I'm infertile. That makes a lot more sense. See, chat? Look at that. That's why. Can't get pregnant. So you might as well fuck whoever you want. Dating older women is called hag maxing now? Why can't men just be normal? Y'all wild. Everyone's wild. Y'all wild. No, no, no, no, no. It's not misogynistic. The term hag here is endearing. We're taking back the term hag. It's like when black people took back the n-word. There's something serious.
Start: 00:37:12 - End: 00:37:39
 No more Call of Duty. No more Call of Duty lobbies. Those are some crazy times. Those are some crazy times. Anybody remember like 2012? I think that's when Call of Duty popped out. No, no, no, no, no, no, no. What was it? It was 2010. It was, wait, no, 2009. I'm thinking of like Call of Duty Mono Warfare 2, where every other word was the N word. You know, we've changed a lot, Chad. And don't even get me started on like YouTube stuff.
Start: 00:37:40 - End: 00:38:09
 Okay, so what's 2009? See, I know my shit. I know my shit. I know when kids squealing and I still think that they probably still do that shit. But like it was that and then like YouTube stuff. Listen, I may not have known very much for Twitch, but YouTube was like 2016. 2016 is when you started seeing a lot of that shit. I don't know. Crazy times. Crazy times. There are different times. Most definitely. Seriously disgusting about men taking advantage of women's loneliness for their resources. To be honest, I don't think the N-word stuff really changed very much. So when it's a younger man dating an older woman...
Start: 00:38:10 - End: 00:38:39
 The younger man is still the predatory one? What? You guys just cannot win. Really? I feel bad for that because it's the whole double standard. It's the whole double standard of it's like, well, I can do this here because I have fucking tits and a vagina. It's like, no, that makes zero sense. You know, the whole equality thing that everyone fucking wants. You know what I mean? The whole equality thing. Yeah, apparently that doesn't apply. It's like you think it does, but then it's like, oh, just kidding. Just kidding.
Start: 00:38:40 - End: 00:39:04
 Like, I still remember that story about there was a teacher that was like having sex with some kid. And like, I think they made a South Park episode about that where he's like, nice, nice. It was so fucking stupid. But have it the other way around. Yeah, it's not gonna, it's not gonna work out all that well. I'm gonna be real with you. It's not gonna work out all that well. Yeah, just saying. Funny episode though, most definitely.
Start: 00:39:06 - End: 00:39:26
 thank you so much for the 13 months by the way thank you okay so we've heard from the hags now let's hear from the hag maxers why are these young men dating older women i have mommy issues oh well that tends to be the one that i see over and over and over again it's like i have mommy issues any mommy issue enjoyers here
Start: 00:39:28 - End: 00:39:58
 Really, I could care less. Just kidding. Just kidding. Just kidding. Just kidding. I don't want to say it. A granny fetish. I'm sorry. I'm going to keep pausing this. This is funny. I like this. Does anyone have a granny fetish? Y'all call me your fucking abuela. So maybe that's what it is. It's like, fuck the mom thing. It's like the abuela. See, it makes more sense now. It makes a lot more sense now. Okay. Okay. That's cool. That's cool.
Start: 00:39:59 - End: 00:40:25
 And I have mommy issues. So what happened? Why do I have so many mommy issues? Like I have mine. That's like a whole different thing. Sometimes older women are just better. The reason why I like older people a lot of the times is because a lot of that drama, a lot of the things that people fucking bitch and complain about, the social media fucking brain rot twats, like all those little bitches.
Start: 00:40:26 - End: 00:40:52
 A lot of times, I'm not saying that older women do not deal with that, but a lot of the times it's less of a thing. We tend to kind of just have already gotten past that. Sometimes. Some people are still in that shit. You can ask some of my old friends from back. Listen, ask some of the other fucking fellow ghetto babies. I like to call them. Some of them are still doing the same old bullshit. Same old fucking bullshit. I know sometimes they're on Facebook. Some. But a lot of the times.
Start: 00:40:53 - End: 00:41:18
 kind of gotten off of that we're kind of not doing the same stupid usually not partying a lot of the times and well i have a couple of friends that still do but it's a lot less let's just say that it's just a lot less so and not only that but like you kind of already have your together kind of have your together especially if you're a single if you are single no kids single no kids a lot of the times like that is like the most desirable thing ever
Start: 00:41:19 - End: 00:41:42
 now ones that already have a family with kids and stuff like that uh some people are like that's too much but ones that actually like have no kids that they're doing well yeah let me just say she's fucking she's fucking she is she's she's fucking it's because then people don't feel like they're like oh i need to be here in this family they just they just fuck
Start: 00:41:44 - End: 00:42:12
 flirty than Zoomer girls. Honestly, it's partly because modern Western girls from 18 to 25 are like 80% hoes now. A woman who is 30 today likely has fewer partners, sexual or other, than a woman who is five years younger than her. Not true. It's probably worse. Especially if she's single. No husband, no nothing. Yeah, no. No. At least from the people that I've known. Yeah, I was gonna say, I'm like, don't wanna...
Start: 00:42:13 - End: 00:42:40
 the reason why he's probably seeing that is because a lot of those girls are very open about that they're talking about it it's the whole empowering thing do you remember that remember that they're like yeah i can do what i want you can't tell me what to do now a lot of older women i i listen i can't say this enough in your 30s that's when you're in your prime your prime sexual like that like that is when people want to have sex the most so
Start: 00:42:40 - End: 00:43:03
 I change my Tinder age range from 20-25 to 30-39.
Start: 00:43:03 - End: 00:43:31
 And have been going on more dates than ever before with beautiful women who do not care that I don't make six figures at 22 years old. Usually because we're established. Take care of a younger woman who we know is potentially a flight risk or appreciate an older woman who has their own shit together. Exactly. George W. Bush married an older woman to get that money. I don't think George W. Bush should be a role model for anything. Didn't he have money though? His family is like set. His dad was the president. Like, no, that doesn't make sense.
Start: 00:43:31 - End: 00:44:00
 Yeah, whoever wrote that, I don't think he did enough research. I don't think he did enough research. Yeah, I was about to say, I'm like, I don't, I don't know. Is that a Reddit thing? I've never been on Reddit. I don't, I don't understand. I don't know how that works. I don't know how that works. I've heard a lot of dumb shit. That's all I gotta say. Yeah, but I thought he had money already. Yeah, I was gonna say, oil tycoons. See, that's what I thought. Because then you gotta worry about babies.
Start: 00:44:00 - End: 00:44:24
 I'm 16 and I'm not attracted to anyone below 30 years old. Oh, come on. You need to live life a little bit. That's the definition of mommy issues. No, he wants to get taken care of, plays video games, and fucks some beautiful 30-year-old. Now, that's a rare occasion. That's a rare occasion. Especially 16. No. Like most people that I know in their 30s.
Start: 00:44:24 - End: 00:44:50
 especially some of my ho friends which again if you want to be a ho you want to be a ho i could care less um but like i mean listen slay um but usually it's in their mid-20s 24 that's what i've noticed 22 23 24 sometimes 25 26 is kind of good like pushing a little bit to where it's like the normal dating but yeah 24 that's what i've noticed maybe in like their mid-20s but like
Start: 00:44:51 - End: 00:45:20
 The reason why I say about 24 is because they've been 21. They've already had their drinking phase. Not saying that they're not still doing it, but like already had at least three years of it. They've kind of gotten a little bit more established with their work, their life. So usually it's around that. Usually. Now, of course, changes via every person. But that's what I've noticed. 24. 24. Ah, yes. The elusive reverse pedophile. Dating a woman with a career, property, experience in bed and life.
Start: 00:45:20 - End: 00:45:46
 What's not to like? Young sluts want the money up front. Old sluts are just happy to get some attention. I was already laughing. True. True. True and real. True and real. Bars. Bars. Legit amazing. I'm 26 now dating a 38 year old after years of trying with women my age and younger. They're all social media obsessed narcissists. I feel like they're hyping up
Start: 00:45:47 - End: 00:46:14
 millennial women too much here oh they're not narcissists obsessed with social media you guys weren't around during the myspace days exactly and listen i still have a lot of people again people my age around 31 32 34 they just do it on facebook that's all it is they do it on facebook they're like look at me doing this i'm just like it's the same thing just a little different it's just a little different different um how should i say i'm
Start: 00:46:14 - End: 00:46:41
 different things that they're posting but a lot of the time it's still the same fucking thing and they're still gossiping and doing all that but i feel like um the younger generation younger generation of women tend to um it's hard to explain it's hard to explain it's just let's just say that's before i'm going into too much detail is that they just do things differently especially for socials they tend to think about it more often let's just say that
Start: 00:46:42 - End: 00:47:11
 But then again, it varies on each person. But that's usually what I notice. Yep, clout. Benson, clout. 100%. When I think older people, I think more in their clout and their friend group. Clout and their friend group, if that makes sense. So a little bit smaller. While I feel like younger folk is more like, hey, clout everywhere. They want people to know you're the baddest bitch. So for me personally, I feel like it is a little different. Little different, but kind of the same. She's 100% correct.
Start: 00:47:12 - End: 00:47:41
 People are still using Facebook. It's huge. Listen, I still have mine. I just never use it. People want to get taken care of. Alright, get out of here. I'm sorry, I can't. I can't with these comments. Who's making these fucking comments? Where is this? It's on YouTube. Wait, is this on YouTube? Twitter? No, no, no, it has to be on fucking
Start: 00:47:42 - End: 00:48:10
 youtube has to be who are these people you get a chat it's like your abuela you don't get it you don't get it you people are sick that's what i'm saying it's happening more than ever it's like 60 some years old unless you just didn't take care of your teeth or that gen z women are too obsessed with drama i didn't want to stop her again but i have to ask has anyone ever done that with someone
Start: 00:48:11 - End: 00:48:38
 It's like the grapefruit, except it's just with her... You didn't have to say that. Okay, if anyone doesn't know about the grapefruit thing, like... All right, you know what? You know what? Actually, I ask these questions, but I'm like, do you really want to know? I'm like, no. Savvy, you really don't want to know. You don't want to know. You don't want to know. That's like a 60-some-year-old usually. That's like how my grandmama is. She has no teeth. Stay away from my grandma.
Start: 00:48:39 - End: 00:48:57
 a lot of men are dropping out of college. The issue is a lot of young college-educated women want college-educated men. See, Stacey is looking for someone college-educated who is making more than her. But not that many young men are on that level yet. Older men are, but not many around Stacey's age. But you know who doesn't care?
Start: 00:48:58 - End: 00:49:26
 Karen. Karen doesn't care about your lack of college degree. Karen wants to pick you up in her 2013 Toyota minivan. Exactly. Bang you in the back seat, then take you to Olive Garden. Oh my god. I have so many stories about... Never mind. Never mind. Never mind. I don't want to talk about it. Listen, me and my ex were very... But... Okay, listen. But it's true though. But it's true though. You do your stuff. Boom. Done. It's like, hey, let's go eat. Nope. And then no extra shit. No extra shit. Sometimes...
Start: 00:49:26 - End: 00:49:55
 Sometimes you could talk about like relationships. So a lot of times like, hey, let's go eat afterwards. Completely just, it's simple. It's simple. It is just simple. And that's like the best part I like to say with like older women, with just adults, with just adults. A lot of the times you can kind of just like, boom, do your thing, go eat, relax. And then just like, that's it. Like, you're not looking at Rebecca who's like really young and she's just like, so what are we now? It's like, sometimes you kind of just do stuff and then you might hang out again.
Start: 00:49:56 - End: 00:50:11
 Who knows? Who knows? Who knows? But like a lot of people just don't really think about it. And if you really like them, then it might happen that way. So I don't know. It varies. It varies again per couple. By the way, Chad, I'm not telling you fucking jack shit. I said what I said. I said what I said.
Start: 00:50:13 - End: 00:50:40
 For basically all of human history, the most prevalent age gap was older men and younger women. But why? Well, it's because men are all disgusting creeps who should be thrown into work camps and castrated and forced to make us pumpkin spice lattes. But also it's because younger women are more fertile. And apparently older people like to do spawn camping. They like to wait patiently for the 18 year old or some people don't. I'm looking at you.
Start: 00:50:40 - End: 00:51:10
 Dr. D. Well, actually in Diddy and apparently this year is just the year of people just talking to kids. It's just strange. Strange. Strange. ...is to make babies, or at least it used to be. Spawn camping. I like that term. Just waiting. That's the whole point. Just waiting. I don't know what the point of modern relationships are anymore. I just learned recently that people purposely get into short term relationships, which is bizarre to me. Like, what's the point? As well as situationships and polycules. The world is very confusing.
Start: 00:51:10 - End: 00:51:39
 all it is really with that is that people just want to fuck they don't want to be held down so they use all these stupid ass terms use all these stupid ass terms to be like hey i want to have sex with you but at the same time you mean pre-ordering beers i heard that earlier today and i was like oh my god please no i like that though it's so stupid people like god damn you um people like to um god damn this look look what you did
Start: 00:51:41 - End: 00:52:10
 Hold on, pause. People like to put all these fancy terms and all these things where it's like, oh my God, we can do this much and this much. But really they just want to fucking see different kind of people. Now with older people, it's not complicated. If that's what they want, people are just like, yeah, well, this is what I like. People just in general, don't mind saying it like it is. A lot of the times when you're little, when you are smaller, when you are young, a lot of the times you do all these terms, not hurt anybody.
Start: 00:52:10 - End: 00:52:39
 But a lot of times when you're older, you kind of just don't care. You're like, well, this is like, it's like they'll try to make it complicated. But really, older people just don't really make it complicated a lot of the times. Again, not everybody, but that's the good majority from what I've seen. It turns out people are different. Pre-ordering. Now, let's be real. Hag Maxing simply does not make any sense from a evolutionary biological standpoint. Thank you for the nine months, by the way. How you doing? But it's 2024. We have surpassed the need for evolutionary biology. We could grow babies and pods and also keep ourselves in them.
Start: 00:52:40 - End: 00:53:09
 Is this true? No way. No fucking way. Wait, no shot. What about the baby and like the... No. No fucking way. Listen, chat says yes. No fucking way. That's crazy. We are starting to go into the future now. And yet we still haven't been able to clone ourself. Listen, that one pisses me off. I'm trying to have multiple me's. Listen, I got shit to do.
Start: 00:53:10 - End: 00:53:39
 I got shit to do and this one self is getting kind of old. So I ain't got as much energy. It's like Futurama shit. Absolutely. Also, another thing with the whole baby thing, our economy fucking sucks. Our economy fucking sucks. And to have a family now, shit, I, I've already been there, done that. You know how much money that is? It's having even one kid is expensive as shit. I had two kids and let me tell you something. That was, it's expensive.
Start: 00:53:40 - End: 00:54:05
 So you think you want to go do other stuff? It's like, how much money do you think you have? You better be a fucking doctor. You better be a fucking doctor. You better be something that's going to make bank because listen, that shit is expensive. That shit is expensive. Not to mention the house and where you're living and oh, don't get me wrong. Cause here's the thing. Cause then you want to put your kids in a good school because schools fucking suck nowadays. So you want to put them in a private school. Also, these stories are my issues.
Start: 00:54:06 - End: 00:54:28
 these are all the stories that me and my ex had to deal with we were just like okay i want to put our kids in the private school because like it was long story as long as the point is is that it's expensive it's expensive and in this economy it's pretty shitty hard to make families now our brains are full of plastic now men wanting to be with younger women made sense i don't know what that means we're the goal but
Start: 00:54:28 - End: 00:54:54
 babies don't seem to be the goal anymore for many older women who simply don't want to have them and for many younger gen z men who think they can't afford them or even worse are scared that having kids will contribute to climate change it's like every other week i see i don't understand that one i don't understand that one also the microplastics i have also have no idea what that is
Start: 00:54:56 - End: 00:55:23
 We live in a crazy world. We live in a crazy world. I don't get it. I don't understand. But at the same time, though, I understand the whole kid thing. Like even for me, again, after my divorce stuff, I was thinking, do I want kids again? Do I want to do all that? I like to say yes. But at the same time, though, the whole thing, the whole thing about it, how you started, what you're doing, how much that it's going to cost, not to mention, like,
Start: 00:55:24 - End: 00:55:53
 if anyone's ever been through a divorce, that shit is expensive. Like, like, and when they say like splitting your money, yeah, no. And you know, for me, I'm kind of old school. I kind of like to like, you know, be with somebody like, I'm like, Hey, I kind of like you. So, you know, I'm very old school when it comes to that. So yeah, it can be shit really fucks with you. So then it's like thinking like, Hey, can I do that shit again? It's like, I don't know. I don't know. And seeing stuff like this really makes you think, really makes you think like, yeah, I don't know about that one, chief.
Start: 00:55:54 - End: 00:56:21
 So the whole older women not wanting kids stuff. It's definitely a thing. It's definitely a thing. It's starting to become a thing. Or more like a thing. Well, we ain't saving... Listen, listen. You know how you make... You want to know how you're saving the planet? Less people. And if we're already not having kids...
Start: 00:56:22 - End: 00:56:50
 We're already not having kids. Listen, it's slowly slimming it out. So we'll be fine. We'll be fine. Just got to just keep not having kids. Listen, Japan does it really well. So we need to start doing the whole Japan thing. Like that works out real well. Can't afford kids. Why does being with a fertile woman even matter? Might as well date an older woman. Might as well scramble those. I mean, am I wrong? Rotten eggs. Oh, why you gotta say it like that? Why you gotta say it like that? Edit that out. Edit that out. No.
Start: 00:56:51 - End: 00:57:20
 Hi, Blair. Thank you for the rave, by the way. Thank you, thank you. Welcome to our Reacting Nanny streams. I hope you had a great stream. Thank you so much for stopping by. In fact, there was a very famous hagmaxer in American history who wrote an entire letter of why it's preferable to sleep with an older woman. And that hagmaxer's name? Benjamin Franklin. President? Mr. Presidente. There's no way. There's no way. But yeah, I hope you enjoyed, um, it's the Void game, right?
Start: 00:57:21 - End: 00:57:47
 Yeah, listen, I gotta look into that. I think the first time I seen that was with Boone. But yeah, enjoy our reacting stream. It's great to have you. Thanks for bringing your awesome community by. Thank you. But yes, Mr. Benjamin Franklin. All right, I gotta hear this. No, I'm not kidding. Benjamin Franklin literally wrote about the benefits of hagmaxing. There ain't no way. And what's funny is a lot of the reasons Benjamin Franklin gives are the same reasons a lot of the Zoomers who hagmax give today.
Start: 00:57:47 - End: 00:58:16
 Because as they have more knowledge of the world and their minds are better stored with observations, their conversation is more improving and more lastingly agreeable. 100% Blair. Because when women cease to be handsome, they study to be good. So basically she can't rely on her looks anymore and has to develop a personality. We talked about that with the last video. The last video of the whole about like the pretty like privileged thing. Listen, for my ladies out there. Fuck, actually for my men out there.
Start: 00:58:16 - End: 00:58:43
 All I got to say is this, please, please, please make it so that when we're talking, hell, even when we're having sex and afterwards or beforehand, where it's like foreplay stuff, if I'm not, all right, I had this ex that I was like seeing, we weren't really fully dating, but I specifically remember she was talking to me about some bullshit and all I wanted to do was leave. I'm like, you know what? Actually, I think I did. I think I did.
Start: 00:58:44 - End: 00:59:12
 Like at first I was kind of in the mood and I was like, all right, I kind of want to do this. Called her over, boom. And then she started talking about some dumb shit. And then I realized I was like, hmm, maybe we're not compatible. Maybe we're not compatible. So then I was like, hey, listen, I got something to do. You know what I did? I had my friend call me. I was like, hey, say that like somebody died or something. Say somebody died. I got, listen, I got to get out of here. Get a personality, get a personality.
Start: 00:59:13 - End: 00:59:37
 Make it so you're not like stick in the mud. Make it so I'm not trying to like have sex with Plank from Etta and Nettie. Please, please, please. I don't care how hot you are. I don't care how hot you are. She was pretty, she was pretty hot. But like the point is, is that I'm not trying to fucking deal with someone's fucking basic ass life. Make it so I actually like you. Give me something. Personality, personality. Get one, Chad. Touch grass.
Start: 00:59:37 - End: 01:00:07
 You don't like plank? I'm not trying to fuck a piece of board. So stupid. He was kind of a pimp, though. They're easier to cheat on your wife with. Lovely. So, funny story with my ex-wife. Okay, actually, that's for another day. We'll talk about that later. It involved all of that, too. Also, Paul, thank you for the... Thank you for the rain.
Start: 01:00:08 - End: 01:00:34
 I don't think I ever talked about that story, but we'll talk about that later. If not, the fucking video is going to be like a thousand hours long. Every animal that walks upright, the deficiency of the fluids that fill the muscles appears first in the highest part. This girl is spitting bars. The face first grows length and wrinkled, then the neck, then the breast and arms. No, no, no, Paul. You came at the perfect time. Thanks so much for stopping by. Welcome to our Reacting Andy content. It's good to have you. Benjamin Franklin really just said, the ass is still fat.
Start: 01:00:35 - End: 01:00:57
 hell yeah brother incredible stuff from our founding fathers i will literally never be able to look at a picture of this man the same again this is no longer the guy who discovered electricity no this is the great american hag maxer he knew what he was listen he knew what he wanted he knew what he wanted he came i like him listen nothing is nothing's more sexier to me as someone that is up front
Start: 01:00:58 - End: 01:01:25
 It is upfront. He knows what he likes. He knows what he likes. And that's the thing that I think a lot of older women like. So actually, no, no other way around. I think that's why the whole like younger generation likes older women is because a lot of the times we know what we want. A lot of the times we make it simple. It's like, hey, I want this. I want this. Boom, done and done. There's no weird shit in between. It's just boom. That's it. It's nice when life is simple, when you're upfront.
Start: 01:01:25 - End: 01:01:49
 Nobody likes that fucking pussy flitting around stupid shit. Just, it's like, no, no, no. Upfront. Upfront and fucking personal. Nice and easy. So basically some Gen Z guys have found a hack in the male loneliness epidemic by dating the women who are suffering from the female loneliness epidemic. Like the two unwanted ends of a bread loaf coming together. And don't even get me started with moms.
Start: 01:01:51 - End: 01:02:20
 Moms and kids. Now, I know for a fact, not everyone wants to be around someone that has already had kids. You know, the ones that already had a save file, even though you should probably like delete that save file. But I'm not going to tell you how to do all that. But point is, because they are. They are probably the most loneliest there. Let's just say that. Make a sandwich. Beautiful. Two people who believe they are the leftovers of society. Two people using each other to temporarily numb the pain.
Start: 01:02:21 - End: 01:02:50
 Or maybe it's not that deep and it's just a funny meme. To be honest, I don't think this is an epidemic, but the internet makes it seem like it is. All the data basically says that women prefer to date men slightly older than them or around the same age as them. And men tend to date women younger than them or around the same age as them. Large age gaps, for the most part, are rare. In relationships where the man is older, he's usually only older by one to two years. In relationships where the woman is older, two to three years. See, I like this. This is what I was trying to talk about.
Start: 01:02:50 - End: 01:03:19
 Like once you get to a certain spot, like hell with my code mistake, this is how far we were about six years. So it was about five to six years with me, my ex-wife there. And we had problems. So it's easier. And usually you have a bit, you have a better chance if it's like just within a few years, by the way. Nice. Nice. Once you start getting lower and lower, Dr. Disrespect, I'm looking at you. It tends to maybe not work.
Start: 01:03:20 - End: 01:03:40
 after this point i think to myself um it's this is sex this is sex this is sex this is a possibility but mean mainly sex um this is like a 50 50 and this is more realistic pretty much oh and i went too far and that's the puppy in all western countries only eight percent of couples have an age gap of 10 or more
Start: 01:03:40 - End: 01:04:07
 And couples with a larger age gap have more of a chance of getting a divorce. But it is a rising trend to think about and keep an eye on. As the social landscape continues to change, so will gender dynamics. And I find that very interesting. But that's enough of my yapping. What do you guys think? What do you think of hagmaxing? Have you ever hagmaxed? Would you hagmax? Personally, I am married to someone younger than me. He was graduating middle school while I was going into college.
Start: 01:04:07 - End: 01:04:31
 so you are okay we must be similar ages actually you might be older than me listen i thought i was older than you all right she's got to be like 35 then give or take give or take i actually don't know her age 34 35 that's my guess oh she's 33 close enough close enough i was i was i was close enough close enough yeah like i said i'm 31 also is that the puppy it's totally a puppy
Start: 01:04:32 - End: 01:05:01
 I like her. She looks like she's in her mid-twenties. I did not know him when he was in middle school. I am not married to a middle schooler. Not a groomer, folks. Do not put in the newspaper that I am a groomer. If you like this video, please consider supporting me on Patreon. Link down below. Thank God for that. Thank God for that. So, so much. Also, big shout out to Holtskern again for sponsoring this video. I have learned so much today. Check them out. Link down below. And I will see you next time. I learned so much today. Bye. By the way, she's adorable. She's super cute.
Start: 01:05:01 - End: 01:05:21
 um i've learned a lot today hag maxine had no idea what the fuck that was i had no idea what the fuck that was chat has been wanting me to watch this for a second um i think the moment that this came out popped inside of like popped in discord and they're like listen you gotta watch this so i like this i like this um
Start: 01:05:23 - End: 01:05:51
 Let me say this because she was saying like, how do you feel like, you know, the climate, the climate of it all? Like, how do you think it's going to change? Well, first off, I think this has always been a thing, but I feel like as of late, it's been getting stronger and stronger and stronger. How's it going to be in the future? That's a great question. That's a great question. I don't know. I want to say that this is going to probably to progress more and more. Let's just say that eventually no one's going to be dating anyone their age.
Start: 01:05:52 - End: 01:06:21
 It's going to be one way all the way over to one side and one way all the way over to the other side. Now, is that good or bad? It's hard to say. But one thing I will say is, well, first off, the whole baby thing, that's starting to get thrown right out the window. I'm not saying throw your baby out the window. Please don't do that if you have kids. But like, I do think to myself, like, hey, that's starting to become less of a thing now, whether it's the economy, whether it's like how people see each other nowadays, the newer generations, etc., etc.,
Start: 01:06:21 - End: 01:06:46
 So I like to say it's just only going to get worse. I think less kids and people are going to be dating way on the opposite end of things. Is it a bad thing? Probably not, especially for this economy. Less people, less people to have to feed. So that's nice. That's nice. But yeah, really cool video. I like this one. So hope you like this one. This is a learned a lot, Chad. And especially when she said something about VTubers. Looking at you, Chad. Looking at you.
"""

In [2]:
prompt = """
You are a specialized AI assistant focused on analyzing and extracting information from transcript texts. Your task is to:

1. Identify and preserve personal experiences and anecdotes that illustrate key points
2. Capture practical insights and real-world implications
3. Include specific examples that demonstrate broader trends
4. Note how different perspectives interact and conflict
5. Provide comprehensive coverage of main ideas, supporting points, and meaningful examples
6. Capture nuanced discussions and multiple perspectives presented 
7. Include relevant context that helps understand the broader topic
8. Extract and organize key talking points in a clear, hierarchical structure
9. Format the output in Markdown using:
   - # for main headings and ## for subheadings
   - **bold** for emphasis on key points
   - *italics* for book titles, scientific names, or foreign words
   - > for notable quotes
   - - or * for bullet points in lists
   - `code blocks` for technical terms or specific jargon
   - --- for section breaks if needed
10. Include timestamp references where relevant
11. Maintain the original meaning and context without adding external information
12. If multiple speakers are present, clearly identify them using their names followed by a colon (e.g., "Speaker: ")

Your output should follow this structure:

## Overview
[Thorough summary of main topic/themes and their significance, including real-world context]

## Key Discussion Points
[Detailed coverage of major points with specific examples and personal experiences]

## Real-World Examples & Experiences
[Personal anecdotes, specific situations, and practical examples shared in discussion]

## Practical Insights
[Actionable insights, observations about real-world dynamics, practical implications]

## Cultural & Social Analysis
[How different groups view/experience the topic, generational differences, cultural shifts]

## Supporting Arguments & Context
[Evidence, reasoning, and background information that enriches understanding]

## Notable Quotes & Moments
[Important quotes that capture key insights or illustrate main points]

## Source Information
- Duration: [Start time] to [End time]
- Format: [Audio/Video transcript]
- Speakers: [List of identified speakers]

Guidelines:
- Prioritize capturing real experiences and practical insights over theoretical discussion
- Include specific examples and anecdotes that illustrate broader points
- Note how personal experiences inform different perspectives
- Preserve important context and nuance from real-world situations
- Highlight practical implications and real-world applications
- Capture generational and cultural differences in perspectives
- Document how different viewpoints interact and conflict
- Use clear, hierarchical markdown formatting
- Include relevant timestamps for major points
- Maintain factual accuracy to the source material
- Indicate any instances where context is assumed or implied
- Use bullet points and sub-bullets for clarity
- Quote directly when necessary for accuracy
- Organize content into logical sections with clear headings

Please process the transcript and organize the information according to these guidelines, emphasizing real-world experiences and practical insights throughout the summary.
"""

In [3]:
from IPython.display import display, Markdown
from summarize_media.summarize_transcription.summarize import get_summarization

summary = get_summarization(text, prompt)

display(Markdown(summary))

## Overview
A detailed analysis of "hag maxing" - a modern dating trend where younger men (particularly Gen Z) seek relationships with older women (typically millennials). The discussion explores societal shifts, economic factors, and changing relationship dynamics.

## Key Discussion Points
- **Definition of Hag Maxing**: Young men (typically Gen Z) actively pursuing romantic relationships with older women
- **Historical Context**: Reference to Benjamin Franklin's writings supporting relationships with older women
- **Age Gaps**: Most successful relationships have 1-3 year gaps; larger gaps (10+ years) represent only 8% of couples
- **Economic Factors**: 
  - Modern housing costs and economy making traditional family formation difficult
  - Financial stability of older women being attractive to younger men
  - Cost of raising children becoming prohibitive

## Real-World Examples & Experiences
- Speaker shares personal experience with 6-year age gap in previous marriage
- Discusses challenges of different life stages in age-gap relationships
- References military experience with age-gap relationships
- Mentions "COVID mistake" relationship with 6-year age difference

## Practical Insights
- **Benefits of Dating Older Women**:
  * Less drama and social media obsession
  * Clear communication about wants/needs
  * Financial stability
  * Established lifestyle
- **Challenges**:
  * Different life stages
  * Varying energy levels
  * Potential fertility issues
  * Cultural differences

## Cultural & Social Analysis
- **Generational Differences**:
  * Social media usage and importance
  * Attitudes toward relationships
  * Financial expectations
  * Views on having children
- **Double Standards**: Discussion of society's different reactions to older men/younger women vs. older women/younger men relationships

## Notable Quotes
> "A lot of times when you're older, you kind of just don't care. You're like, well, this is what I like."

> "The whole equality thing that everyone fucking wants... apparently that doesn't apply."

## Source Information
- Duration: Approximately 1 hour 6 minutes
- Format: Video reaction/commentary
- Speakers: Main reactor and video content creator

## Gradio Demo

In [ ]:
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown(
    """
    # Media Summarizer
    
    Input the youtube video that you want to transcribe + summarize
    
    Wait time: 5-10 mins (depends on )
    
    Feedback: [Google form](https://forms.gle/RT66bre2X9B9s4d79) or dm me (twitter(X):shivvor2 or discord: shz2__)
    """
    )
    input = gr.Textbox(label = "url", placeholder = "Please input a valid youtube url") 
    sampling_rate = gr.Slider(value=16000, minimum=8000, maximum=24000, step=100, label = "Audio Sample Rate")
    out_summary = gr.Markdown(label = "summary")
    out_transcript = gr.Textbox(label = "transcript")
    start_btn = gr.Button("Un-NotebookLM's your podcast")
    start_btn.click(fn = pipeline, 
                    inputs = input, 
                    outputs = [out_summary, out_transcript]
                    )

demo.launch()


INFO:httpx:HTTP Request: GET http://127.0.0.1:7860/gradio_api/startup-events "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: HEAD http://127.0.0.1:7860/ "HTTP/1.1 200 OK"


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


INFO:httpx:HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"


Title: HagMaxxing - the new Hypergamy
Upload date: 07/10/24 13:00:13
Time length: 1:06:48


INFO:summarize_media.host_files.host_files:Uploading file: HagMaxxing - the new Hypergamy.wav


Converted audio size: 244.6 MB
Exceeded size limit, uploading to file host before performing inference


INFO:summarize_media.host_files.host_files:Successfully uploaded to https://0x0.st/Xdf-.wav


Uploaded File
Starting transcription process


INFO:httpx:HTTP Request: POST https://api.replicate.com/v1/predictions "HTTP/1.1 201 Created"
INFO:httpx:HTTP Request: GET https://api.replicate.com/v1/models/victor-upmeet/whisperx/versions/84d2ad2d6194fe98a17d2b60bef1c7f910c46b2f6fd38996ca457afd9c8abfcb "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
